<a name="top"></a>
<div style="width:1000 px">

<div style="float:right; width:98 px; height:98px;">
<img src="https://cdn.miami.edu/_assets-common/images/system/um-logo-gray-bg.png" alt="Miami Logo" style="height: 98px;">
</div>
    
<div style="float:left; width:120 px; height:120px;">
<img src="http://seasonedchaos.github.io/assets/img/SC_logo.jpg" alt="Seasoned Chaos logo" style="height: 120px;">
</div>

<div style="clear:both"></div>
</div>
    
<h1>Satellite Pull and Plot: Integrated Water Vapor</h1><h3><i>Python code by: Kayla Besong, Seasoned Chaos team</i></h3>
<br>
<h3>Objective: To select historical range of integrated water vapor satellite images from Space Science and Engineering Center
University of Wisconsin - Madison and output a gif. The date range as of 2022/02 is currently from 2016/11-2021/12. This is expected to change as time progresses and it is encouraged you visit <a href= http://tropic.ssec.wisc.edu/real-time/mtpw2/> their website</a> to obtain the most up-to-date information.</h3> 
<br>
<br>
<i>last major edit: 01/30/2022


<hr style="height:2px;">

In [ ]:
### required packages ###

import ftplib
import pandas as pd
from io import BytesIO
from PIL import Image, ImageDraw

In [ ]:
def pull_plot(region, start_date, end_date):
    
    
    """ This function takes the input region, start date, and end date, obtains the satellite 
        data from the ssec.wisc.edu ftp server and generates an output .gif saved to the local 
        machine. It does not store files temporarily on local machine. 
    
        region: string, input region from bulleted list below
        start_date: string of date to start loop featuring day, month, and year in any order using either / or - 
        end_date: string of date to start loop featuring day, month, and year in any order using either / or -     
    
    """
    
    
    
    date_range = pd.date_range(start = start_date, end = end_date, freq = 'H')                 ## establish date range given inputs 
    months = pd.unique([i.strftime('%Y%m') for i in date_range])                               ## create unique Year/Month array to loop through ftp directory based on years and months
                                                                                               ## this is somewhat of a saftey net if event desires straddles months or years
    
    ftp = ftplib.FTP('ftp.ssec.wisc.edu', 'anonymous')                                         ## log into the FTP server

    images = []                                                                                ## create empty array to store files obtained from looping through ftp directories

    for yearmon in months:                                                                     ## loop through unique years/months desired
            
        try:
       
            ftp.cwd(f"/pub/mtpw2/images/tpw_nrl_colors/{region}/{yearmon}")                    ## cd into correct directory based on region and year/month desired
        
            for pic in sorted(ftp.nlst()):                                                     ## ftp.nlst lists all the files in our ftp directory accessed above, we are looping over each image in this list

                r = BytesIO()                                                                  ## establish an in-memory bytes buffer for us to temporarily store the ftp image in

                sub = pd.to_datetime(str(pic)[4:15], format = '%Y%m%d.%H')                     ## ensure the image selected is within date range

                if sub >= date_range[0] and sub <= date_range[-1]:

                    print(f'{sub} is downloading')

                    ftp.retrbinary(f'RETR {pic}', r.write)                                     ## retrive the image from the list and store it in our established byte buffer

                    images.append(r)                                                           ## append the byte buffer for later plotting
        
        except:

            print('Invalid region or date range inputted')
            
        
    img, *imgs = [Image.open(f) for f in images]                                               ## formally open each temporarily stored byte buffer in images list
    
    img.save(f'IWV_{region}_{str(date_range[0])[0:10]}_{str(date_range[-1])[0:10]}.gif',       ## utilize image gif plotter that accepts output name (first row here is the saved output name based on region and dates)
             format='GIF', append_images=imgs, save_all=True, duration=200, loop=3)            ## and append_images = our list of opened images that are in byte form
                                                                                               ## gif features can be altered here such as # of loops and speed     

    ftp.quit()                                                                                 ## make sure to close our connection!! 



## A current list of regions availble:

- alaska     
- ausf     
- conus
- epac
- europe
- global    (Atlantic centric)
- global2   (Pacific centric)
- indo
- namer     (North America) 
- natl      (Atlantic ocean only)
- samer1    (Northern half of South America)
- samer2    (Southern half of South America)
- spac      (Southern Pacific 110E-110W) -- only available 2021/08 onwards
- wpac

NOTE: It is *VERY* important to input the region directly as is above. Using 'alaska' works, however 'Alaska' does not.

## Based on Start Date and End Date

Each date requires a day, month, and year. The format and order is not highly sensitive but does require a - or / to divide the times. Pandas date_range is used here, so whatever is accepted by pandas is accepted here.

In [ ]:
## example of inputs for pull_plot function
## note how start and end date 

region = 'alaska'
start_date = '2018-12-31'
end_date = '2019-01-01'

In [ ]:
%%time

pull_plot(region, start_date, end_date)